In [21]:
!tree -d

.
├── images
│   ├── test
│   │   ├── JPEGImages
│   │   └── Visualization
│   ├── train
│   │   ├── JPEGImages
│   │   └── Visualization
│   └── val
│       ├── JPEGImages
│       └── Visualization
├── pretrained_models
│   ├── faster_rcnn_inception_v2_coco_2018_01_28
│   │   └── saved_model
│   └── mask_rcnn_inception_v2_coco_2018_01_28
│       └── saved_model
├── scripts
└── training

17 directories


In [5]:
# vamos a dividir nuestras imágenes y anotaciones en splits de train, test y validación.
# los argumentos -vr y -tr indican respectivamente los porcentajes de datos reservados para validación y test
!python3 scripts/train_test_split.py -i images -o images -tr 0.1 -vr 0.15

succesfully created validation split using:  15.0 % of data
succesfully created test split using:  10.0 % of data
succesfully created train split using:  75.0 % of data


In [13]:
# vamos a convertir estas fotos en y anotaciones en datasets de formato COCO. Tenemos que hacerlo para los sets de train
# test y validación. Tenemos que pasar como argumento archivo txt con los labels siguiendo el formato indiado en la
# documentación oficial de labelme https://github.com/wkentaro/labelme/tree/master/examples/instance_segmentation. Un ejemplo
# de cómo debe ser este archivo se muestra debajo
!python3 scripts/labelme2coco.py images/train images/train/coco --labels training/labels.txt
!python3 scripts/labelme2coco.py images/test images/test/coco --labels training/labels.txt
!python3 scripts/labelme2coco.py images/val images/val/coco --labels training/labels.txt

Creating dataset: images/train/coco
Generating dataset from: images/train/images_frame2000.json
Generating dataset from: images/train/images_frame8050.json
Generating dataset from: images/train/images_frame2750.json
Generating dataset from: images/train/images_frame7900.json
Generating dataset from: images/train/images_frame5800.json
Generating dataset from: images/train/images_frame10250.json
Generating dataset from: images/train/images_frame2950.json
Generating dataset from: images/train/images_frame3750.json
Generating dataset from: images/train/images_frame10150.json
Generating dataset from: images/train/images_frame5200.json
Generating dataset from: images/train/images_frame1700.json
Generating dataset from: images/train/images_frame5150.json
Generating dataset from: images/train/images_frame10050.json
Generating dataset from: images/train/images_frame7350.json
Generating dataset from: images/train/images_frame1350.json
Generating dataset from: images/train/images_frame3350.json
G

Generating dataset from: images/val/images_frame1200.json
Generating dataset from: images/val/images_frame1050.json
Generating dataset from: images/val/images_frame9250.json
Generating dataset from: images/val/images_frame1250.json
Generating dataset from: images/val/images_frame4000.json
Generating dataset from: images/val/images_frame3200.json
Generating dataset from: images/val/images_frame9350.json
Generating dataset from: images/val/images_frame2200.json
Generating dataset from: images/val/images_frame7050.json
Generating dataset from: images/val/images_frame4250.json
Generating dataset from: images/val/images_frame2400.json
Generating dataset from: images/val/images_frame7850.json
Generating dataset from: images/val/images_frame8550.json
Generating dataset from: images/val/images_frame4300.json
Generating dataset from: images/val/images_frame5750.json
Generating dataset from: images/val/images_frame4050.json
Generating dataset from: images/val/images_frame3050.json
Generating dat

<img src="training/ejemplo_labels_txt.PNG">

In [11]:
# ahora borramos las imágenes y anotaciones de las carpetas train, test y val, ya que la carpeta coco dentro de cada 
# una de estas carpetas ya contiene las imágenes origina
!rm images/test/*
!mv images/test/coco/* images/test
!rm -rf images/test/coco 

!rm images/train/*
!mv images/train/coco/* images/train
!rm -rf images/train/coco 

!rm images/val/*
!mv images/val/coco/* images/val
!rm -rf images/val/coco 

In [23]:
# ahora tenemos que generar los tfrecords usando estos dataset en formato coco
!python3 object_detection/dataset_tools/create_coco_tf_record.py --logtostderr \
--train_image_dir=images/train \
--val_image_dir=images/val \
--test_image_dir=images/test \
--train_annotations_file=images/train/annotations.json \
--val_annotations_file=images/val/annotations.json \
--testdev_annotations_file=images/test/annotations.json \
--output_dir=training/tfrecords \
--include_masks=True

python3: can't open file 'object_detection/dataset_tools/create_coco_tf_record.py': [Errno 2] No such file or directory
